# LBB: Streamlining Invoice Processing
### Document Understanding Transformer for Efficient Business Operations in Python
 
👩🏻‍🏫 Created By : Heinz Metrosan Donradt
 

### Import Library
Pertama-tama, mari kita import library yang sesuai dengan proyek kita. Jangan lupa setting option yang memudahkan kita melihat output.

In [36]:
import re # untuk cleaning menggunakan regex (regular expression)
import os # untuk pembacaan data image -> baca nama file
import torch

from PIL import Image # untuk membaca gambar
from transformers import DonutProcessor, VisionEncoderDecoderModel

In [82]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Import Model & Processor
Lalu kita tentukan model dan processor yang diinginkan. Saat ini, kita menggunakan model dan processor "naver-clova-ix/donut-base-finetuned-cord-v2"

In [38]:
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [39]:
# set model device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

VisionEncoderDecoderModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0-1): 2 x DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
           

### Import Image ke dalam list
Setelah kita melakukan pencarian gambar struk, kita import ke dalam list

In [40]:
# folder path
dir_path = 'lbb_input/'

In [41]:
img_filenames = [] # list nama file gambar
images = [] # list objek gambar
labels = [] # list label dari gambar

# iterasi direktori -> looping untuk mendapatkan seluruh nama file di `data_input/valid``
for file in os.listdir(dir_path):
    if file.endswith(('.png', '.jpg', '.jpeg')): # jika file tsb berekstensi ('.png', '.jpg', '.jpeg')
        # menyimpan nama gambar ke list
        img_filenames.append(file)
        
        # membuka dan menyimpan objek gambar ke list
        images.append(Image.open(dir_path + file))


    elif file.endswith('.txt'):  # jika file tersebut berekstensi .txt
        # membuka file teks dan membaca isinya, lalu menyimpan ke list
        with open(dir_path + file, 'r') as label_file: 
            labels.append(label_file.read())


### Transform Gambar menjadi Text
Dengan bantuan DONUT, kita mulai men-transformasi-kan gambar yang sudah kita masukkan ke dalam list menjadi text

In [42]:
task_prompt = "<s_cord-v2>"

In [43]:
def doc_to_text(input_img, task_prompt=task_prompt, model=model, processor=processor):
    # set model device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    
    # document preprocessing
    pixel_values = processor(input_img, return_tensors="pt").pixel_values
    decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]

    # sequence generation
    outputs = model.generate(
        pixel_values.to(device),
        decoder_input_ids=decoder_input_ids.to(device),
        max_length=model.decoder.config.max_position_embeddings,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id,
        use_cache=True,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
        
        # modify parameters
        early_stopping=True,
        num_beams=2,
        output_scores=True,
    )
    
    # document post-processing: sequence token cleaning
    sequence = processor.batch_decode(outputs.sequences)[0]
    sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
    sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
    print(sequence)

    # output conversion: token to json
    output = processor.token2json(sequence)

    return output

In [45]:
# list untuk menyimpan hasil generate docs to text
preds = []
 
# step 1: looping gambar yang ada di list `images`
for img in images[:10]:
    # step 2: kita men-generate 1 per 1 dari gambarnya
    result = doc_to_text(img)
    preds.append(result)

<s_menu><s_nm> ++Water 500ml</s_nm><s_cnt> 1</s_cnt><s_price> 0</s_price><sep/><s_nm> Bugsy Fried Rice</s_nm><s_cnt> 1</s_cnt><s_price> 145,000</s_price><sep/><s_nm> Corona Extra</s_nm><s_cnt> 1</s_cnt><s_price> 150,000</s_price><sep/><s_nm> Devils Lair Cab Sav</s_nm><s_cnt> 1</s_cnt><s_price> 3,500,000</s_price><sep/><s_nm> Dunhill Putih</s_nm><s_cnt> 1</s_cnt><s_price> 50,000</s_price><sep/><s_nm> Inniskilin Cabernet</s_nm><s_cnt> 1</s_cnt><s_price> 4,200,000</s_price><sep/><s_nm> Lighter</s_nm><s_cnt> 1</s_cnt><s_price> 15,000</s_price><sep/><s_nm> Macallan 18 Sher Btl</s_nm><s_cnt> 1</s_cnt><s_price> 8,800,000</s_price><sep/><s_nm> Marlboro Ice Blast</s_nm><s_cnt> 1</s_cnt><s_price> 50,000</s_price><sep/><s_nm> Martell XO Btl</s_nm><s_cnt> 1</s_cnt><s_price> 9,200,000</s_price><sep/><s_nm> Mushroom Soup</s_nm><s_cnt> 1</s_cnt><s_price> 65,000</s_price><sep/><s_nm> Salmon Saute</s_nm><s_cnt> 1</s_cnt><s_price> 175,000</s_price><s_sub><s_nm> ++Pokka GreenT 500ml</s_nm><s_cnt> * 2</s_

Setelah result text didapatkan, mari kita ubah text dictionary menjadi dataframe

In [46]:
df_preds = pd.DataFrame.from_dict(preds)
df_preds

,menu,sub_total,total
0,"[{'nm': '++Water 500ml', 'cnt': '1', 'price': ...","{'subtotal_price': '80,925,000', 'service_pric...","{'total_price': '106,214,063', 'menuqty_cnt': ..."
1,"[{'nm': 'CAPPUCCINO', 'price': '45,000'}, {'nm...","{'subtotal_price': '286,000', 'service_price':...","{'total_price': '330,330', 'cashprice': '330.3..."
2,"[{'nm': 'MANSO JUICE', 'cnt': '3', 'price': 'R...","{'subtotal_price': 'Rp 36,759,000', 'tax_price...","{'total_price': 'Rp 37,218,390', 'total_etc': ..."
3,"[{'nm': 'KUE PANADA', 'unitprice': '9.000', 'c...","{'subtotal_price': '76.000', 'tax_price': '7.6...","{'total_price': '87 600', 'menuqty_cnt': '5'}"
4,"[{'nm': 'CUMI CABE HIJAU', 'cnt': '1', 'price'...","{'subtotal_price': '213,500', 'service_price':...","{'total_price': '248,941'}"
5,"[{'nm': 'Kopi Susu Gula Aren Hot', 'cnt': '3',...","{'subtotal_price': '370.000', 'service_price':...","{'total_price': '419,210'}"
6,"[{'nm': 'LUMPIA BASAH (M)', 'cnt': '2', 'price...","{'subtotal_price': '48,000', 'tax_price': '4,3...","{'total_price': '48,000', 'cashprice': '50,000..."
7,"[{'nm': 'ES TEH KANIS JUMBO', 'cnt': '3', 'pri...",{'subtotal_price': '$221.000.00'},{'total_price': '$221.000.00'}
8,"[{'nm': 'CHRTSANTIMUM TEA', 'cnt': '1', 'price...","{'subtotal_price': '71,766,600', 'service_pric...","{'total_price': '86,837,586', 'menutype_cnt': ..."
9,"[{'nm': 'AIR MINERAL', 'cnt': '1', 'price': '1...","{'subtotal_price': '164,000', 'discount_price'...","{'total_price': '119,460', 'menutype_cnt': ':4..."


In [47]:
df_preds.isna().sum()

menu         0
sub_total    0
total        0
dtype: int64

In [48]:
df = pd.DataFrame(columns=['receipt_id', 'nm', 'cnt', 'price', 'total_price'])
df

,receipt_id,nm,cnt,price,total_price


In [49]:
# mengiterasi setiap rrow dalam df_preds
for index, row in df_preds.iterrows():

    # mengecek apakah field 'menu' adalah list, jika tidak, ubah menjadi list
    menus = row['menu'] if isinstance(row['menu'], list) else [row['menu']]

    # mengiterasi setiap elemen dalam list 'menus'
    for menu in menus:
        # menambahkan row baru ke 'df' dengan informasi dari 'menu' serta tambahan 'total_price' dan 'receipt_id'
        df.loc[len(df)] = {
            **menu,  # unpack semua pasangan key-value dari dictionary 'menu' yang cocok dg nama kolom yg ada
            'total_price': row['total']['total_price'],  # mengambil 'total_price' dari baris saat ini
            'receipt_id': img_filenames[index].split('.')[0]  # mengambil ID struk dari nama file gambar
        }

Berikut contoh dataframe yang sudah berhasil kita dapatkan:

In [50]:
df.head()

,receipt_id,nm,cnt,price,total_price
0,ID001,++Water 500ml,1,0,"106,214,063"
1,ID001,Bugsy Fried Rice,1,"145,000","106,214,063"
2,ID001,Corona Extra,1,"150,000","106,214,063"
3,ID001,Devils Lair Cab Sav,1,"3,500,000","106,214,063"
4,ID001,Dunhill Putih,1,"50,000","106,214,063"


Mari kita simpan result dataframe ke dalam csv

In [51]:
# menyimpan DataFrame 'df' ke dalam file csv
df.to_csv('lbb_receipt_extracted.csv', 
          index=False)  # berarti indeks DataFrame tidak disertakan dalam file csv


Sehingga kita tidak perlu mengulang transformasi yang memerlukan waktu yang lama, kita bisa read dari csv yang telah kita simpan di step sebelumnya.

In [61]:
df = pd.read_csv('lbb_receipt_extracted.csv' )

In [63]:
new_columns_name = {
    'nm': 'item_name',
    'cnt': 'quantity'
}

df = df.rename(columns=new_columns_name)
df

,receipt_id,item_name,quantity,price,total_price
0,ID001,++Water 500ml,1.0,0,"106,214,063"
1,ID001,Bugsy Fried Rice,1.0,"145,000","106,214,063"
2,ID001,Corona Extra,1.0,"150,000","106,214,063"
3,ID001,Devils Lair Cab Sav,1.0,"3,500,000","106,214,063"
4,ID001,Dunhill Putih,1.0,"50,000","106,214,063"
...,...,...,...,...,...
96,ID009,LOBST MUTIARA(100GR),35.0,"16,434,000","86,837,586"
97,ID010,AIR MINERAL,1.0,"10,000","119,460"
98,ID010,BEEF TONGUE STEAK,1.0,"98,000","119,460"
99,ID010,STEAK HUT OMELETTE,1.0,"46,000","119,460"


In [64]:
# fungsi untuk membersihkan string harga dari simbol dan spasi
def clean_price(x):
    return float(x.replace(".", "").replace(",", "").replace("Rp", "").replace("$", "").replace(" ", ""))


In [65]:
df

,receipt_id,item_name,quantity,price,total_price
0,ID001,++Water 500ml,1.0,0,"106,214,063"
1,ID001,Bugsy Fried Rice,1.0,"145,000","106,214,063"
2,ID001,Corona Extra,1.0,"150,000","106,214,063"
3,ID001,Devils Lair Cab Sav,1.0,"3,500,000","106,214,063"
4,ID001,Dunhill Putih,1.0,"50,000","106,214,063"
...,...,...,...,...,...
96,ID009,LOBST MUTIARA(100GR),35.0,"16,434,000","86,837,586"
97,ID010,AIR MINERAL,1.0,"10,000","119,460"
98,ID010,BEEF TONGUE STEAK,1.0,"98,000","119,460"
99,ID010,STEAK HUT OMELETTE,1.0,"46,000","119,460"


In [66]:
df.loc[df['price'].str.contains("Rp 696-10"), 'price'] = 'Rp 696,000'
df['price'] = df['price'].apply(lambda x: clean_price(str(x)))

In [67]:
df['total_price'] = df['total_price'].apply(lambda x: clean_price(str(x)))

In [69]:
# membersihkan dan mengkonversi nilai 'x' menjadi float
def clean_quantity(x):
    # jika 'x' bernilai None (tidak ada data), kembalikan 0
    if x is None:
        return 0

    # mengonversi 'x' ke string, menghapus spasi, 'x'/'X', dan ']' dari string tersebut, dan mengembalikan menjadi sebuah angka
    return float(str(x).replace(" ", "").replace("x", "").replace("X", "").replace("]", ""))

In [71]:
# code here: mengaplikasikan fungsi 'clean_quantity' ke setiap elemen di kolom 'quantity' pada 'df'
df['quantity'] = df['quantity'].apply(lambda x: clean_quantity(x))

In [75]:
# menyimpan DataFrame 'df' ke dalam file csv
df.to_csv('lbb_receipt_clean.csv', 
          index=False)  # berarti indeks DataFrame tidak disertakan dalam file csv


In [74]:
# code here
df_pivot = df.pivot_table(index=['receipt_id', 'total_price', 'item_name', 'quantity'])
df_pivot

price
receipt_id total_price item_name            quantity           
ID001      106214063.0 ++Water 500ml        1.0             0.0
                                            2.0             0.0
                       Bihun Goreng Seafood 15.0      1275000.0
                       Bugsy Fried Rice     1.0        145000.0
                                            4.0        550000.0
...                                                         ...
ID009      86837586.0  STM VG CONPOY PORSAL 1.0        111000.0
ID010      119460.0    AIR MINERAL          1.0         10000.0
                       BEEF TONGUE STEAK    1.0         98000.0
                       STEAK HUT OMELETTE   1.0         46000.0
                       TEH MANIS            1.0         10000.0

[95 rows x 1 columns]

### Analisis 1: Item Paling Banyak Dipesan
Menggunakan frequency table untuk mengidentifikasi item yang paling banyak dipesan.

In [76]:
item_frequency = df['item_name'].value_counts()

# Menampilkan 5 item teratas yang paling banyak dipesan
top_items = item_frequency.head(5)
print("Top 5 Most Ordered Items:")
print(top_items)

Top 5 Most Ordered Items:
item_name
++Water 500ml           2
Macallan 18 Sher Btl    2
CAPPUCCINO              2
HAZELNUT CAPPUCCINO     2
Bugsy Fried Rice        2
Name: count, dtype: int64


### Analisis 2: Total Penjualan per Receipt
Menggunakan conditional subsetting untuk menghitung total penjualan per receipt.

In [77]:
# Menghitung total penjualan per receipt
total_sales_per_receipt = df.groupby('receipt_id')['total_price'].sum()

# Menampilkan total penjualan per receipt
print("Total Sales per Receipt:")
print(total_sales_per_receipt)

Total Sales per Receipt:
receipt_id
ID001    3.080208e+09
ID002    1.981980e+06
ID003    7.443678e+08
ID004    4.380000e+05
ID005    1.742587e+06
ID006    2.096050e+06
ID007    1.920000e+05
ID008    1.989000e+08
ID009    1.042051e+09
ID010    4.778400e+05
Name: total_price, dtype: float64


### Analisis 3: Rata-rata Harga Item per Quantity 
Menggunakan conditional subsetting untuk menghitung rata-rata harga item per quantity.

In [80]:
# Menghitung rata-rata harga item per quantity
avg_price_per_quantity = df.groupby('item_name')['price'].mean()

# Menampilkan rata-rata harga item per quantity
print("Average Price per Quantity for Each Item:")
print(avg_price_per_quantity)

Average Price per Quantity for Each Item:
item_name
++Water 500ml                     0.0
AIR MINERAL                   10000.0
AMLOME TSHAMSA 29HU/PC     27800000.0
APPETIZER PLATTER           3456000.0
AVOCADO JUICE                112000.0
BEEF TONGUE STEAK             98000.0
BLACK COFFEE                  76000.0
Bihun Goreng Seafood        1275000.0
Bugsy Fried Rice             347500.0
CAPPUCCINO                    45000.0
CHRTSANTIMUM TEA              12000.0
CUMI CABE HIJAU               34500.0
Coca Cola                    165000.0
Corona Extra                 150000.0
Croissant Cheese              35000.0
DIET COKE                     15000.0
Devils Lair Cab Sav         3500000.0
Dunhill Putih                 50000.0
EDAMAME                      215000.0
EQUIL HATURAL                 38000.0
EQUIL NATURAL                210000.0
EQUIL SPARSOLING              74000.0
ES TEH KANIS JUMBO          1500000.0
Es Jeruk Siosar               70000.0
FRESH LEMON SODA              40000.